In [13]:
#use "topfind"
#require "base";;
open Base;;

- : unit = ()
Findlib has been successfully loaded. Additional directives:
  #require "package";;      to load a package
  #list;;                   to list the available packages
  #camlp4o;;                to load camlp4 (standard syntax)
  #camlp4r;;                to load camlp4 (revised syntax)
  #predicates "p,q,...";;   to set these predicates
  Topfind.reset();;         to force that packages will be reloaded
  #thread;;                 to enable threads

- : unit = ()


/home/demouser/.opam/default/lib/base/caml: added to search path
/home/demouser/.opam/default/lib/base/caml/caml.cma: loaded
/home/demouser/.opam/default/lib/base/shadow_stdlib: added to search path
/home/demouser/.opam/default/lib/base/shadow_stdlib/shadow_stdlib.cma: loaded
/home/demouser/.opam/default/lib/sexplib0: added to search path
/home/demouser/.opam/default/lib/sexplib0/sexplib0.cma: loaded
/home/demouser/.opam/default/lib/base: added to search path
/home/demouser/.opam/default/lib/base/base.cma: loaded


In [14]:
let ratio x y = 
Float.of_int x /. Float.of_int y

val ratio : int -> int -> Base.Float.t = <fun>


Modules can also be opened to make their contents available without explicitly qualifying by the module name. We did that once already, when we opened `Base` earlier. We can use that to make this code a littele easier to read, both avoiding the repetition of `Float` above, and avoiding use of the slightly awkaward `/.` operator. In the following example ,we open the `Float.O` module, which has a bunch of useful operators and functions that are designed to be used in this kind of context. Note that this causes the standard int-only arithmetic operators to be shadowed locally.

In [16]:
let ratio x y = 
let open Float.O in
of_int x / of_int y;;

val ratio : int -> int -> Base.Float.t = <fun>


In [19]:
ratio 4 6

- : Base.Float.t = 0.66666666666666663


Note that we used a slightly different syntax for opening the module, since we only opening it in the local scope inside the difinition of `ratio`. There's also a more concise syntax for local opens

In [21]:
let ratio x y = 
  Float.O.(of_int x / of_int y);;


val ratio : int -> int -> Base.Float.t = <fun>


Over time, you'll build a rought inition for how the Ocaml inference engine works, which makes it easier to reason through your programs. You can also make it easier to understand the types of a given expression by adding explicit type annotations. There annotations don't change the behavior of an Ocaml program, but they can serve as useful documentaiton, as well as catch unintended type changes.

In [22]:
let first_if_true test x y = 
if test x then x else y

val first_if_true : ('a -> bool) -> 'a -> 'a -> 'a = <fun>


Indeed, if we look at the type returned by the toplevel, we see that rather than choose a single concrete type, Ocaml has introduced a `type variable 'a` to express that the type is generic. In particular, the type of the `test` argument is `(a -> bool)`, which means that `test` is one-argument function whose return value is `bool` and whose argument could be any type `'a`. But, whatever type `'a` is, it has to be the same as the type of the other two arguments, `x` and `y`, and of the return value of `first_if_true`. This kind of genericity is called `parametic polymorphism` because it works by parameterizing the type in question with a type variale

In [24]:
let languages = ["French"; "Spanish"; "OCaml"; "Perl"; "C"]

val languages : string list = ["French"; "Spanish"; "OCaml"; "Perl"; "C"]


In [25]:
List.map languages ~f:String.length

- : int Base.List.t = [6; 7; 5; 4; 1]


In [26]:
List.map ~f:String.length languages

- : int Base.List.t = [6; 7; 5; 4; 1]


Notably, the function passed to `List.map` is passed under a `labeled argumetn ~f`. Labeled arguments are specified by name rather than by position, and thus allow you to change the order in which arguments are presented to a function without changing its behavior, as you can see here:

comma creates a tuple, even if there are no surrounding parens.

In [28]:
1,2,3;;

- : int * int * int = (1, 2, 3)


concatenate two lists

In [30]:
[1;2;3]@[4;5;6]

- : int Base.List.t = [1; 2; 3; 4; 5; 6]


In [35]:
let my_favorite_language (my_favorite :: the_rest) = my_favorite

File "[35]", line 1, characters 25-64:
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
[]


val my_favorite_language : 'a list -> 'a = <fun>


the pattern is not exhausive. This means that there are values of type in question that won't be captured by the pattern

In [32]:
my_favorite_language ["English";"Spanish";"French"]

- : string = "English"


In [33]:
my_favorite_language []

error: runtime_error

An option is used to express that a value might or might not be present.

In [37]:
let divide x y = 
if y = 0 then None else Some (x / y)

val divide : Base.Int.t -> Base.Int.t -> Base.Int.t option = <fun>


In [38]:
divide 4 0

- : Base.Int.t option = None


In [42]:
divide 5 2

- : Base.Int.t option = Some 2


Note that `String.rsplit2` has return type `(string * string) option`, returning `None` when no character was found to split on. 

In [41]:
let downcase_extension filename = 
match String.rsplit2  filename ~on:'.' with
| None ->filename
| Some (base, ext) ->
base ^ "." ^ String.lowercase ext

val downcase_extension : Base.String.t -> Base.String.t = <fun>


Options are important because they are the standard way in Ocaml to encode a value that might not be there; there's no such thing as `NullPointerException` in Ocaml. This is different from most other languages, including Java and C#, where most if not all data types are nullable.

In [43]:
type point2d = {x : float; y : float};;

type point2d = { x : Base.float; y : Base.float; }


In [46]:
let p = { x = 3.; y = -4.};;

val p : point2d = {x = 3.; y = -4.}


And we can get access to the contents of these types using pattern matching

In [45]:
let magnitude { x = x_pos; y = y_pos} = Float.sqrt (x_pos **. 2. +. y_pos **. 2.);;

val magnitude : point2d -> Base.Float.t = <fun>


We can write this more tersely using what's called `field punning`. In particular, when the name of the field and the name of the variable it is bound to coincide, we don't have to write them both down.

Alternatively, we can use dot notation for accessing record fields:

In [47]:
let distance v1 v2 = magnitude {x = v1.x -. v2.x; y = v1.y -. v2.y}

val distance : point2d -> point2d -> Base.Float.t = <fun>


In [53]:
{x = 1.0;y = 2.0}

- : point2d = {x = 1.; y = 2.}


And we can of course include our newly defined types as components in larger types.

In [54]:
type circle_desc = {center: point2d; radius: float};;

type circle_desc = { center : point2d; radius : Base.float; }


In [55]:
type rect_desc = {lower_left: point2d; width: float; height: float}

type rect_desc = {
  lower_left : point2d;
  width : Base.float;
  height : Base.float;
}


In [57]:
type segment_desc = {endpoint1: point2d; endpoint2:point2d}

type segment_desc = { endpoint1 : point2d; endpoint2 : point2d; }


Now, imagine that you want to combine multiple objects of these types together as a descriptin of a multi-object scene. You need some unified way of representing these objects together in a single type. One way of doing this is using a `variant type`:

In [58]:
type scene_element = 
| Circle of circle_desc
| Rect of rect_desc
| Segment of segment_desc

type scene_element =
    Circle of circle_desc
  | Rect of rect_desc
  | Segment of segment_desc


Here's how we might write a function for testing whether a point is in the interior of some element of a list of `scene_elements`

In [59]:
let is_inside_scene_element point scene_element = 
let open Float.O in
match scene_element with
|Circle {center; radius} ->
distance center point < radius
| Rect {lower_left; width; height} ->
point.x > lower_left.x && point.x < lower_left.x + width
&& point.y > lower_left.y && point.y < lower_left.y + height
| Segment {endpoint1;endpoint2} -> false

val is_inside_scene_element : point2d -> scene_element -> Base.bool = <fun>


In [60]:
let is_inside_scene point scene = 
List.exists scene
~f:(fun el -> is_inside_scene_element point el)

val is_inside_scene : point2d -> scene_element Base.List.t -> bool = <fun>


In [62]:
is_inside_scene {x=3.;y=7.} [ Circle {center = {x=4.;y= 4.}; radius = 0.5 } ]

- : bool = false


In [65]:
is_inside_scene {x=3.;y=7.}
[ Circle {center = {x=4.;y= 4.}; radius = 5.0 };Circle {center = {x=4.;y= 4.}; radius = 0.5 }]

- : bool = true


In [74]:
 is_inside_scene_element {x=3.;y=7.} (Circle {center = {x=4.;y= 4.}; radius = 0.5 })

- : Base.bool = false


In [75]:
 is_inside_scene_element {x=3.;y=7.} (Circle {center = {x=4.;y= 4.}; radius = 5. })

- : Base.bool = true
